In [9]:
!pip install woma

  Using cached woma-1.4.3-py3-none-any.whl.metadata (12 kB)
  Using cached seagen-1.5-py3-none-any.whl.metadata (3.4 kB)
Using cached woma-1.4.3-py3-none-any.whl (55.3 MB)
Using cached seagen-1.5-py3-none-any.whl (27 kB)


In [1]:
import swiftsimio
from swiftsimio import load
import h5py

In [2]:
!pwd

/home/lacruzen/Desktop/swift-cmgi/Demo3/SpinImpact


In [3]:
!ls /home/lacruzen/Desktop/swift-cmgi/Demo3/SpinImpact

demo_impact_n50.hdf5	plot_snapshots-old.py  statistics.txt
demo_impact_n50.yml	plot_snapshots.py      task_level_0.txt
demo_plots		README.md	       timesteps.txt
dependency_graph_0.csv	restart		       unused_parameters.yml
getICs.sh		run.sh		       used_parameters.yml
output_n50.txt		snapshots	       visualization.ipynb


In [4]:
data = load("/home/lacruzen/Desktop/swift-cmgi/Demo3/SpinImpact/demo_impact_n50.hdf5")

In [5]:
data

SWIFT dataset at /home/lacruzen/Desktop/swift-cmgi/Demo3/SpinImpact/demo_impact_n50.hdf5. 
Available groups: gas

In [6]:
data.gas

SWIFT dataset at /home/lacruzen/Desktop/swift-cmgi/Demo3/SpinImpact/demo_impact_n50.hdf5. 
Available fields: coordinates, densities, density, internal_energies, internal_energy, masses, material_ids, particle_ids, pressures, smoothing_length, smoothing_lengths, velocities

In [7]:
data.gas.velocities

cosmo_array([[ 0.00063595, -0.00024245,  0.        ],
       [ 0.00063595, -0.00024245,  0.        ],
       [ 0.00063595, -0.00024245,  0.        ],
       ...,
       [-0.00424124,  0.00161694,  0.        ],
       [-0.00424124,  0.00161694,  0.        ],
       [-0.00424124,  0.00161694,  0.        ]],
      shape=(118466, 3), dtype=float32, units='(dimensionless)', comoving='True', cosmo_factor='a**0.0 at a=1.0', valid_transform='True')

In [19]:
test = np.linspace(0,54000,28)
test

array([    0.,  2000.,  4000.,  6000.,  8000., 10000., 12000., 14000.,
       16000., 18000., 20000., 22000., 24000., 26000., 28000., 30000.,
       32000., 34000., 36000., 38000., 40000., 42000., 44000., 46000.,
       48000., 50000., 52000., 54000.])

In [31]:
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py
import woma
from matplotlib.patches import Patch

# Plotting setup
plt.rcParams.update({
    "font.size": 12,
    "font.family": "serif",
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12
})

# Earth units
R_E = 6.3710e6  # m

# Simulation parameters
SNAPSHOT_START = 0
SNAPSHOT_END = 27
SNAPSHOT_INTERVAL = 2000  # seconds between snapshots

def load_snapshot(filename):
    """Load and convert the particle data to plot."""
    with h5py.File(filename, "r") as f:
        # Units from file metadata
        file_to_SI = woma.Conversions(
            m=float(f["Units"].attrs["Unit mass in cgs (U_M)"]) * 1e-3,
            l=float(f["Units"].attrs["Unit length in cgs (U_L)"]) * 1e-2,
            t=float(f["Units"].attrs["Unit time in cgs (U_t)"]),
        )

        # Particle data
        A2_pos = (
            np.array(f["PartType0/Coordinates"][()])
            - 0.5 * f["Header"].attrs["BoxSize"]
        ) * file_to_SI.l
        A1_mat_id = np.array(f["PartType0/MaterialIDs"][()])
        
        # Load velocities
        A2_vel = np.array(f["PartType0/Velocities"][()]) * file_to_SI.v

    # Restrict to z < 0 for plotting
    A1_sel = np.where(A2_pos[:, 2] < 0)[0]
    A2_pos = A2_pos[A1_sel]
    A1_mat_id = A1_mat_id[A1_sel]
    A2_vel = A2_vel[A1_sel]

    return A2_pos, A1_mat_id, A2_vel

def separate_target_impactor(positions, velocities, material_ids):
    """
    Separate target and impactor particles based on material IDs and positions.
    More robust separation using both material distribution and spatial clustering.
    """
    # Method 1: Use material ID distribution (if materials are different)
    unique_materials, material_counts = np.unique(material_ids, return_counts=True)
    
    if len(unique_materials) >= 2:
        # If we have at least 2 materials, assume they're distributed between bodies
        print(f"Materials found: {unique_materials} with counts: {material_counts}")
        
        # Simple heuristic: larger group is target, smaller is impactor
        if material_counts[0] > material_counts[1]:
            target_mask = material_ids == unique_materials[0]
            impactor_mask = material_ids == unique_materials[1]
        else:
            target_mask = material_ids == unique_materials[1]
            impactor_mask = material_ids == unique_materials[0]
    else:
        # Method 2: Spatial separation based on density clustering
        print("Using spatial separation...")
        distances = np.linalg.norm(positions, axis=1)
        
        # Find natural break in distance distribution
        hist, bin_edges = np.histogram(distances, bins=50)
        cumulative = np.cumsum(hist) / np.sum(hist)
        
        # Find where cumulative distribution reaches ~50%
        split_index = np.argmax(cumulative > 0.5)
        split_distance = bin_edges[split_index]
        
        target_mask = distances < split_distance
        impactor_mask = ~target_mask
    
    print(f"Target particles: {np.sum(target_mask)}")
    print(f"Impactor particles: {np.sum(impactor_mask)}")
    
    return target_mask, impactor_mask

def calculate_bulk_properties(positions, velocities, mask):
    """Calculate bulk properties for a group of particles."""
    if np.sum(mask) == 0:
        return None
    
    group_positions = positions[mask]
    group_velocities = velocities[mask]
    
    # For bulk velocity, use simple average (since we don't have masses in your loading function)
    bulk_velocity = np.mean(group_velocities, axis=0)
    bulk_speed = np.linalg.norm(bulk_velocity)
    
    # Velocity dispersion
    velocity_dispersion = np.std(group_velocities, axis=0)
    dispersion_magnitude = np.linalg.norm(velocity_dispersion)
    
    # Center of mass (simple average)
    center_of_mass = np.mean(group_positions, axis=0)
    
    return {
        'bulk_velocity': bulk_velocity,
        'bulk_speed': bulk_speed,
        'velocity_dispersion': velocity_dispersion,
        'dispersion_magnitude': dispersion_magnitude,
        'center_of_mass': center_of_mass,
        'n_particles': np.sum(mask),
        'mean_position': center_of_mass
    }

def calculate_simulation_time(snapshot_id):
    """Calculate simulation time for a given snapshot ID."""
    return SNAPSHOT_START + (snapshot_id * SNAPSHOT_INTERVAL)

def format_time(seconds):
    """Format time in seconds to a human-readable string."""
    hours = seconds / 3600
    minutes = (seconds % 3600) / 60
    secs = seconds % 60
    
    if hours >= 1:
        return f"{hours:.1f} h"
    elif minutes >= 1:
        return f"{minutes:.1f} min"
    else:
        return f"{secs:.0f} s"

def plot_velocity_analysis(A2_pos, A1_mat_id, A2_vel, snapshot_id):
    """Create comprehensive velocity visualization for a single snapshot."""
    time = calculate_simulation_time(snapshot_id)
    time_str = format_time(time)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'Impact Simulation - Time: {time_str} (Snapshot {snapshot_id})', fontsize=16)
    
    # Separate target and impactor
    target_mask, impactor_mask = separate_target_impactor(A2_pos, A2_vel, A1_mat_id)
    
    # Calculate bulk properties
    target_props = calculate_bulk_properties(A2_pos, A2_vel, target_mask)
    impactor_props = calculate_bulk_properties(A2_pos, A2_vel, impactor_mask)
    
    # Plot 1: Velocity magnitude scatter plot
    velocity_magnitudes = np.linalg.norm(A2_vel, axis=1)
    sc1 = axes[0,0].scatter(A2_pos[:,0] / R_E, A2_pos[:,1] / R_E, 
                           c=velocity_magnitudes, cmap='viridis', s=2, alpha=0.8)
    axes[0,0].set_xlabel('r"$x$ ($R_\oplus$)")')
    axes[0,0].set_ylabel('r"$y$ ($R_\oplus$)")')
    axes[0,0].set_title('Velocity Magnitude (m/s)')
    axes[0,0].set_aspect('equal')
    axes[0,0].grid(True, alpha=0.3)
    plt.colorbar(sc1, ax=axes[0,0], label='Velocity (m/s)')
    
    # Plot 2: Colour by material
    Di_mat_colour = {"ANEOS_Fe85Si15": "darkgray", "ANEOS_forsterite": "orangered"}
    Di_id_colour = {woma.Di_mat_id[mat]: colour for mat, colour in Di_mat_colour.items()}

    A1_colour = np.empty(len(A2_pos), dtype=object)
    for id_c, c in Di_id_colour.items():
        A1_colour[A1_mat_id == id_c] = c
    
    axes[0,1].scatter(A2_pos[:, 0] / R_E, A2_pos[:, 1] / R_E,
        c=A1_colour, edgecolors="none", marker=".", alpha=0.5
    )
    # ax_lim = 10
    # axes[0,1].set_xlim(-ax_lim, ax_lim)
    # axes[0,1].set_ylim(-ax_lim, ax_lim)
    axes[0,1].set_xlabel(r"$x$ ($R_\oplus$)")
    axes[0,1].set_ylabel(r"$y$ ($R_\oplus$)")
    axes[0,1].set_title("Material Composition")
    axes[0,1].set_aspect('equal')
    axes[0,1].grid(True, alpha=0.3)
    
    # colors = np.zeros(len(A2_pos))
    # colors[target_mask] = 1
    # colors[impactor_mask] = 2
    
    # sc2 = axes[0,1].scatter(A2_pos[:,0] / R_E, A2_pos[:,1] / R_E, 
    #                        c=colors, cmap='tab10', s=2, alpha=0.8)
    # axes[0,1].set_xlabel('X position ($R_\oplus$)')
    # axes[0,1].set_ylabel('Y position ($R_\oplus$)')
    # axes[0,1].set_title('Target (blue) vs Impactor (red)')
    # axes[0,1].set_aspect('equal')
    # axes[0,1].grid(True, alpha=0.3)
    
    # # Add legends for the color coding
    # legend_elements = [
    #     plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=8, label='Target'),
    #     plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=8, label='Impactor')
    # ]
    # axes[0,1].legend(handles=legend_elements, loc='upper left')
    
    # Plot 3: Velocity vectors (subsampled for clarity)
    subsample = slice(None, None, 50)  # Plot every 50th particle
    speed = np.linalg.norm(A2_vel[subsample], axis=1)
    quiver = axes[0,2].quiver(A2_pos[subsample,0] / R_E, A2_pos[subsample,1] / R_E,
                             A2_vel[subsample,0], A2_vel[subsample,1],
                             scale=1e4, color='red', alpha=0.6, width=0.003)
    axes[0,2].set_xlabel('X position ($R_\oplus$)')
    axes[0,2].set_ylabel('Y position ($R_\oplus$)')
    axes[0,2].set_title('Velocity Vectors (subsampled)')
    axes[0,2].set_aspect('equal')
    axes[0,2].grid(True, alpha=0.3)
    
    # Plot 4: Velocity distributions
    if np.sum(target_mask) > 0 and np.sum(impactor_mask) > 0:
        target_velocities = np.linalg.norm(A2_vel[target_mask], axis=1)
        impactor_velocities = np.linalg.norm(A2_vel[impactor_mask], axis=1)
        
        axes[1,0].hist(target_velocities, bins=30, alpha=0.7, label='Target', color='blue', density=True)
        axes[1,0].hist(impactor_velocities, bins=30, alpha=0.7, label='Impactor', color='red', density=True)
        axes[1,0].set_xlabel('Velocity Magnitude (m/s)')
        axes[1,0].set_ylabel('Probability Density')
        axes[1,0].set_title('Velocity Distribution')
        axes[1,0].legend()
        axes[1,0].grid(True, alpha=0.3)
    
    # Plot 5: Velocity components distribution
    axes[1,1].hist(A2_vel[:,0], bins=30, alpha=0.7, label='Vx', color='orange', density=True)
    axes[1,1].hist(A2_vel[:,1], bins=30, alpha=0.7, label='Vy', color='green', density=True)
    axes[1,1].hist(A2_vel[:,2], bins=30, alpha=0.7, label='Vz', color='purple', density=True)
    axes[1,1].set_xlabel('Velocity Components (m/s)')
    axes[1,1].set_ylabel('Probability Density')
    axes[1,1].set_title('Velocity Components Distribution')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # Plot 6: Bulk properties summary
    axes[1,2].axis('off')
    summary_text = []
    
    if target_props:
        summary_text.extend([
            "TARGET PROPERTIES:",
            f"Bulk speed: {target_props['bulk_speed']:.2f} m/s",
            f"Particles: {target_props['n_particles']}",
            f"Vel dispersion: {target_props['dispersion_magnitude']:.2f} m/s",
            f"Position: ({target_props['mean_position'][0]/R_E:.2f}, " +
                      f"{target_props['mean_position'][1]/R_E:.2f}) R⊕"
        ])
    
    if impactor_props:
        summary_text.extend([
            "",
            "IMPACTOR PROPERTIES:",
            f"Bulk speed: {impactor_props['bulk_speed']:.2f} m/s",
            f"Particles: {impactor_props['n_particles']}",
            f"Vel dispersion: {impactor_props['dispersion_magnitude']:.2f} m/s", 
            f"Position: ({impactor_props['mean_position'][0]/R_E:.2f}, " +
                      f"{impactor_props['mean_position'][1]/R_E:.2f}) R⊕"
        ])
    
    if target_props and impactor_props:
        relative_velocity = np.linalg.norm(target_props['bulk_velocity'] - impactor_props['bulk_velocity'])
        separation = np.linalg.norm(target_props['mean_position'] - impactor_props['mean_position'])
        summary_text.extend([
            "",
            "RELATIVE MOTION:",
            f"Relative speed: {relative_velocity:.2f} m/s",
            f"Separation: {separation/R_E:.2f} R⊕"
        ])
    
    if summary_text:
        axes[1,2].text(0.05, 0.95, '\n'.join(summary_text), transform=axes[1,2].transAxes,
                      fontfamily='monospace', fontsize=10, verticalalignment='top',
                      bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.7))
    
    plt.tight_layout()
    return fig, target_props, impactor_props

def analyze_velocity_evolution():
    """Analyze velocity evolution across all snapshots."""
    # Storage for time evolution data
    time_series = {
        'times': [],
        'target_bulk_speed': [],
        'impactor_bulk_speed': [], 
        'relative_speed': [],
        'target_dispersion': [],
        'impactor_dispersion': [],
        'separation': []
    }
    
    output_dir = "velocity_analysis"
    os.makedirs(output_dir, exist_ok=True)
    
    print("Starting velocity evolution analysis...")
    print("=" * 60)
    
    for snapshot_id in range(SNAPSHOT_END + 1):
        filename = f"snapshots/demo_impact_n50_{snapshot_id:04d}.hdf5"
        time = calculate_simulation_time(snapshot_id)
        
        if not os.path.exists(filename):
            print(f"Snapshot {snapshot_id:02d} not found: {filename}")
            continue
            
        print(f"Processing snapshot {snapshot_id:02d}, time = {format_time(time)}")
        
        try:
            # Load data using your function
            A2_pos, A1_mat_id, A2_vel = load_snapshot(filename)
            
            # Create individual snapshot analysis
            fig, target_props, impactor_props = plot_velocity_analysis(
                A2_pos, A1_mat_id, A2_vel, snapshot_id
            )
            
            # Save individual snapshot
            plt.savefig(f"{output_dir}/velocity_snapshot_{snapshot_id:04d}.png", 
                       dpi=150, bbox_inches='tight')
            plt.close()
            
            # Store time series data
            if target_props and impactor_props:
                time_series['times'].append(time)
                time_series['target_bulk_speed'].append(target_props['bulk_speed'])
                time_series['impactor_bulk_speed'].append(impactor_props['bulk_speed'])
                
                relative_velocity = np.linalg.norm(
                    target_props['bulk_velocity'] - impactor_props['bulk_velocity']
                )
                time_series['relative_speed'].append(relative_velocity)
                
                time_series['target_dispersion'].append(target_props['dispersion_magnitude'])
                time_series['impactor_dispersion'].append(impactor_props['dispersion_magnitude'])
                
                separation = np.linalg.norm(
                    target_props['mean_position'] - impactor_props['mean_position']
                )
                time_series['separation'].append(separation / R_E)
                
                print(f"  ✓ Target: {target_props['bulk_speed']:.1f} m/s, " +
                      f"Impactor: {impactor_props['bulk_speed']:.1f} m/s, " +
                      f"Relative: {relative_velocity:.1f} m/s")
            else:
                print(f"  ⚠ Could not separate bodies properly")
                
        except Exception as e:
            print(f"  ✗ Error processing snapshot {snapshot_id}: {e}")
            continue
    
    # Create time evolution summary plot
    if time_series['times']:
        print(f"\nCreating time evolution plots...")
        create_evolution_plots(time_series, output_dir)
    
    print("=" * 60)
    print(f"Analysis complete! Results saved to: {output_dir}/")

def create_evolution_plots(time_series, output_dir):
    """Create comprehensive time evolution plots."""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Impact Simulation - Velocity Evolution', fontsize=16)
    
    times = np.array(time_series['times'])
    
    # Plot 1: Bulk speeds evolution
    axes[0,0].plot(times, time_series['target_bulk_speed'], 'b-', 
                  label='Target', linewidth=2.5, marker='o', markersize=4)
    axes[0,0].plot(times, time_series['impactor_bulk_speed'], 'r-', 
                  label='Impactor', linewidth=2.5, marker='s', markersize=4)
    axes[0,0].set_xlabel('Time (s)')
    axes[0,0].set_ylabel('Bulk Speed (m/s)')
    axes[0,0].set_title('Bulk Speeds Evolution')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Plot 2: Relative speed and separation
    ax2 = axes[0,1]
    color = 'green'
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Relative Speed (m/s)', color=color)
    ax2.plot(times, time_series['relative_speed'], color=color, 
            linewidth=2.5, marker='^', markersize=4)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.grid(True, alpha=0.3)
    
    ax2_twin = ax2.twinx()
    color = 'purple'
    ax2_twin.set_ylabel('Separation (R⊕)', color=color)
    ax2_twin.plot(times, time_series['separation'], color=color, 
                 linewidth=2.5, linestyle='--', marker='d', markersize=4)
    ax2_twin.tick_params(axis='y', labelcolor=color)
    
    axes[0,1].set_title('Relative Motion')
    
    # Plot 3: Velocity dispersions
    axes[1,0].plot(times, time_series['target_dispersion'], 'b--', 
                  label='Target dispersion', linewidth=2, marker='o', markersize=3)
    axes[1,0].plot(times, time_series['impactor_dispersion'], 'r--', 
                  label='Impactor dispersion', linewidth=2, marker='s', markersize=3)
    axes[1,0].set_xlabel('Time (s)')
    axes[1,0].set_ylabel('Velocity Dispersion (m/s)')
    axes[1,0].set_title('Internal Velocity Dispersions')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Plot 4: Speed ratios and trends
    speed_ratio = np.array(time_series['impactor_bulk_speed']) / np.array(time_series['target_bulk_speed'])
    axes[1,1].plot(times, speed_ratio, 'purple', linewidth=2.5, marker='*', markersize=5)
    axes[1,1].axhline(y=1.0, color='gray', linestyle=':', alpha=0.7, label='Equal speed')
    axes[1,1].set_xlabel('Time (s)')
    axes[1,1].set_ylabel('Speed Ratio (Impactor/Target)')
    axes[1,1].set_title('Speed Ratio Evolution')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/velocity_evolution_summary.png", dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Evolution plots saved to {output_dir}/velocity_evolution_summary.png")

if __name__ == "__main__":
    # Test with a single snapshot first
    test_snapshot = 0
    test_filename = f"snapshots/demo_impact_n50_{test_snapshot:04d}.hdf5"
    
    if os.path.exists(test_filename):
        print("Testing with single snapshot...")
        A2_pos, A1_mat_id, A2_vel = load_snapshot(test_filename)
        fig, _, _ = plot_velocity_analysis(A2_pos, A1_mat_id, A2_vel, test_snapshot)
        plt.savefig("test_velocity_analysis.png", dpi=150, bbox_inches='tight')
        plt.close()
        print("✓ Test analysis saved as test_velocity_analysis.png")
    
    # Run full analysis
    analyze_velocity_evolution()

Testing with single snapshot...
Materials found: [400 402] with counts: [41492 17725]
Target particles: 41492
Impactor particles: 17725
✓ Test analysis saved as test_velocity_analysis.png
Starting velocity evolution analysis...
Processing snapshot 00, time = 0 s
Materials found: [400 402] with counts: [41492 17725]
Target particles: 41492
Impactor particles: 17725
  ✓ Target: 73.7 m/s, Impactor: 75.7 m/s, Relative: 2.0 m/s
Processing snapshot 01, time = 33.3 min
Materials found: [400 402] with counts: [41477 17731]
Target particles: 41477
Impactor particles: 17731
  ✓ Target: 91.6 m/s, Impactor: 92.5 m/s, Relative: 9.2 m/s
Processing snapshot 02, time = 1.1 h
Materials found: [400 402] with counts: [41470 17753]
Target particles: 41470
Impactor particles: 17753
  ✓ Target: 193.7 m/s, Impactor: 143.1 m/s, Relative: 174.8 m/s
Processing snapshot 03, time = 1.7 h
Materials found: [400 402] with counts: [41453 17754]
Target particles: 41453
Impactor particles: 17754
  ✓ Target: 289.4 m/s, 